In [ ]:
import re
import json
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://drawdownga.gatech.edu/files/2024/08/data-centers-ver6.html"
driver.get(url)

driver.implicitly_wait(10)

page_source = driver.page_source

soup = BeautifulSoup(page_source, 'html.parser')

# Find the script tag with type "application/json" and data-for attribute
script_tag = soup.find('script', type='application/json', attrs={'data-for': re.compile('htmlwidget-*')})

if script_tag:
    # Extract the JSON data
    json_text = script_tag.string

    # Parse the JSON data
    data = json.loads(json_text)

    # Navigate through the JSON structure to find the data
    # data['x']['calls'][...], we need to find the method 'addCircleMarkers' and extract its arguments
    calls = data['x']['calls']

    for call in calls:
        if call['method'] == 'addCircleMarkers':
            args = call['args']
            latitudes = args[0]  # First argument is a list of latitudes
            longitudes = args[1]  # Second argument is a list of longitudes
            names = args[3]  # Fourth argument is a list of names
            break
    else:
        print("Could not find 'addCircleMarkers' in the JSON data.")
        driver.quit()
        exit()

    # Ensure that the lengths match then create dataframe
    if len(latitudes) == len(longitudes) == len(names):
        df = pd.DataFrame({
            'Name': names,
            'Latitude': latitudes,
            'Longitude': longitudes
        })

        # Save data to a CSV file or display it
        df.to_csv('data_centers_extracted.csv', index=False)
        print(df.head()) 
    else:
        print("Mismatch in lengths of extracted data")

else:
    print("Could not find.")

# Close the driver
driver.quit()

                               Name   Latitude  Longitude
0    X.AT4..Atlanta.IBX.Data.Center  33.895445 -84.453611
1           X365.Smyrna.Data.Center  33.841159 -84.522407
2           AT2.Atlanta.Data.Center  33.926009 -84.481383
3                          Coloblox  33.883949 -84.472318
4  DC.Blox.Atlanta.ATL1.Data.Center  33.831712 -84.339108
